In [3]:
# Import libraries

from sqlalchemy import Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base, relationship, mapper
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd 
import json
from datetime import datetime
import datetime
import sqlite3
from sqlite3 import Error
from bs4 import BeautifulSoup

# Extract

Extract json data from .txt file

In [2]:
with open('database.txt') as f:
    json_data = json.load(f)   

# Load

Load the data to the relational database

In [3]:
# Create a declarative base class
Base = declarative_base()

Create table game_dev

In [4]:
# table game_dev
game_dev = Table('game_dev', Base.metadata,
    Column('steam_appid', Integer, ForeignKey('games.steam_appid')),
    Column('developer_id', Integer, ForeignKey('developers.developer_id'))
)

Create table games

In [5]:
# table games
class Game(Base):
    __tablename__ = 'games'

    steam_appid = Column(Integer, primary_key=True)
    game_type = Column(String)
    game_name = Column(String)
    required_age = Column(Integer)
    is_free  = Column(String)
    detailed_description = Column(String)
    about_the_game = Column(String)
    short_description = Column(String)
    header_image = Column(String)
    website = Column(String)
    pc_requirements_min = Column(String)
    pc_requirements_recomm = Column(String)
    mac_requirements_min = Column(String)
    mac_requirements_recomm = Column(String)
    price_currency = Column(String)
    price_initial = Column(Integer)
    price_final = Column(Integer)
    price_discount_percent = Column(Integer)
    price_initial_formatted = Column(String)
    price_final_formatted = Column(String)
    release_date_coming_soon = Column(String)
    release_date = Column(String)
    support_info_url = Column(String)
    support_info_email = Column(String)
    support_info_background = Column(String)
    num_reviews = Column(Integer)
    review_score = Column(Integer)
    total_positive = Column(Integer)
    total_negative = Column(Integer)
    total_reviews = Column(Integer)
    platforms_windows = Column(String)
    platforms_mac = Column(String)
    platforms_linux = Column(String)
    #devs = relationship("Dev", secondary=game_dev)

Create table developers

In [6]:
# table developers
class Dev(Base):
    __tablename__ = 'developers'
    
    developer_id = Column(Integer, primary_key=True, autoincrement=True)
    developer = Column(String)
    #games = relationship("Game", secondary=game_dev, lazy = "select", backref="developer")
    games = relationship("Game", secondary=game_dev)

Create table game_genre

In [7]:
# table game_genre

game_genre = Table('game_genre', Base.metadata,
    Column('steam_appid', Integer, ForeignKey('games.steam_appid')),
    Column('genre_id', Integer, ForeignKey('genres.genre_id'))
)

Create table genres

In [8]:
#table genres

class Genre(Base):
    __tablename__ = 'genres'
    
    genre_id = Column(Integer, primary_key=True)
    genre_descript = Column(String)
    #games = relationship("Game", secondary=game_dev, lazy = "select", backref="developer")
    games = relationship("Game", secondary=game_genre)

Create table game_cat

In [9]:
# table game_cat

game_cat = Table('game_cat', Base.metadata,
    Column('steam_appid', Integer, ForeignKey('games.steam_appid')),
    Column('category_id', Integer, ForeignKey('categories.category_id'))
)

Create table categories

In [10]:
#table categories

class Category(Base):
    __tablename__ = 'categories'
    
    category_id = Column(Integer, primary_key=True)
    category_descript = Column(String)
    #games = relationship("Game", secondary=game_dev, lazy = "select", backref="developer")
    games = relationship("Game", secondary=game_cat)

In [11]:
engine = create_engine('sqlite:///steam_games.db')

In [12]:
print(type(engine))

<class 'sqlalchemy.engine.base.Engine'>


In [13]:
# Create database following parameters defined on `Base`:
Base.metadata.create_all(engine)

In [14]:
# Bind the engine to the metadata of the Base class so that the
# declaratives can be accessed through a DBSession instance
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

Insert data - games

In [15]:
def get_game(game):
    return session.query(Game).filter_by(steam_appid = game['steam_appid']).first()

In [16]:
def insert_game(game):
    db_game = Game(steam_appid = game['steam_appid'], 
                   game_type = game['type'], 
                   game_name = game['name'], 
                   required_age = game['required_age'], 
                   is_free = game['is_free'],
                   detailed_description = game['detailed_description'], 
                   about_the_game = game['about_the_game'], 
                   short_description = game['short_description'], 
                   header_image = game['header_image'],
                   website = game['website'],
                   pc_requirements_min = game['pc_requirements']['minimum'] if 'pc_requirements' in game and 'minimum' in game['pc_requirements'] else '',
                   price_currency = game['price_overview']['currency'] if 'price_overview' in game and 'currency' in game['price_overview'] else '',
                   price_initial = game['price_overview']['initial'] if 'price_overview' in game and 'initial' in game['price_overview'] else 0,
                   price_final = game['price_overview']['final'] if 'price_overview' in game and 'final' in game['price_overview'] else 0,
                   price_discount_percent = game['price_overview']['discount_percent'] if 'price_overview' in game and 'discount_percent' in game['price_overview'] else 0,
                   price_initial_formatted = game['price_overview']['initial_formatted'] if 'price_overview' in game and 'initial_formatted' in game['price_overview'] else '',
                   price_final_formatted = game['price_overview']['final_formatted'] if 'price_overview' in game and 'final_formatted' in game['price_overview'] else '',
                   release_date_coming_soon = game['release_date']['coming_soon'] if 'release_date' in game and 'coming_soon' in game['release_date'] else '',
                   release_date = game['release_date']['date'] if 'release_date' in game and 'date' in game['release_date'] else '',
                   support_info_url = game['support_info']['url'] if 'support_info' in game and 'url' in game['support_info'] else '',
                   support_info_email = game['support_info']['email'] if 'support_info' in game and 'email' in game['support_info'] else '',
                   support_info_background = game['background'],
                   num_reviews = game['num_reviews'],
                   review_score = game['review_score'],
                   total_positive = game['total_positive'],
                   total_negative = game['total_negative'],
                   total_reviews = game['total_reviews'],
                   platforms_windows = game['platforms']['windows'] if 'platforms' in game and 'windows' in game['platforms'] else '',
                   platforms_mac = game['platforms']['mac'] if 'platforms' in game and 'mac' in game['platforms'] else '',
                   platforms_linux = game['platforms']['linux'] if 'platforms' in game and 'linux' in game['platforms'] else ''
                  )
    session.add(db_game)
    session.commit()         
    return db_game
    

In [17]:
def add_game(game):
    db_game = get_game(game)
    if db_game is None:
        db_game = insert_game(game)
    return db_game    

Insert data developers

In [18]:
def get_developer(developer):
    return session.query(Dev).filter_by(developer = developer).first()

In [19]:
def insert_developer(developer):
    db_dev = Dev(
        developer = developer
                  )
    session.add(db_dev)
    session.commit()       
    return db_dev

In [20]:
def add_developer(developer, db_game):
    print(developer)
    db_dev = get_developer(developer)
    if db_dev is None:
        db_dev = insert_developer(developer)
    db_dev.games.append(db_game)
    session.commit()       

Insert data - genres

In [21]:
def get_genre(genre):
    return session.query(Genre).filter_by(genre_id = genre['id']).first()

In [22]:
def insert_genre(genre):
    db_genre = Genre(genre_id = genre['id'], 
                   genre_descript = genre['description'])
    session.add(db_genre)
    session.commit()         
    return db_genre

In [23]:
def add_genre(genre, db_game):
    db_genre = get_genre(genre)
    if db_genre is None:
        db_genre = insert_genre(genre)
    db_genre.games.append(db_game)
    return db_genre    

Insert data - categories

In [24]:
def get_category(category):
    return session.query(Category).filter_by(category_id = category['id']).first()

In [25]:
def insert_category(category):
    db_category = Category(category_id = category['id'], 
                   category_descript = category['description'])
    session.add(db_category)
    session.commit()         
    return db_category

In [26]:
def add_category(category, db_game):
    db_category = get_category(category)
    if db_category is None:
        db_category = insert_category(category)
    db_category.games.append(db_game)
    return db_category

In [27]:
for game in list(json_data.values()):
    print(game['name'])
    db_game = add_game(game)
    if 'developers' in game: 
        for developer in game['developers']:
            add_developer(developer, db_game)
    if 'genres' in game: 
        for genre in game['genres']:
            add_genre(genre, db_game)
    if 'categories' in game: 
        for category in game['categories']:
            add_category(category, db_game)                

All That Remains: Part 1
Glitch Games
Paperball
Cliax Games
Moderium
All Saints Gaming
Pizza Time Explosion
Electric Prune Juice
Customer Cums First!
Miel
Chained: A Victorian Nightmare
MWM Interactive
Hentai Police
Podval Games
KremlinDev
Colony Siege
Finifugal Games
Wolf and Pigs
Visual Light
Monster Girl Garden
Noxious Games
Dogworld
Lateralis
Slave Girl Reno
Aphrodite
Danger City
Hash Technology
哈视奇科技
The Button Witch
DDreams Games
Space Block Buster
Reality MagiQ
Mr.King Luo!Don't be kidding
艾斯貝洛
koloro
Summoner VR
Broadway Entertainment Group Interactive
COWBOY YAKUZA
Active Gaming Media Inc.
Kawaii Girl
MoeGame
Rhythm Fighter
echo games
The Burning Descent
RyseUp Studios
The Secret Order 8: Return to the Buried Kingdom
Sunward Games
Shoot, push, portals
CrewLab
Braverz
Evil Noob Studios
Wide Ocean Big Jacket
Turnfollow
War Remains: Dan Carlin Presents an Immersive Memory
Flight School Studio
Christmas Stories: Enchanted Express Collector's Edition
Elephant Games
The Christmas Sp

Posibility
UGtobest
Moonlight Basket
Inu to Neko
Yuna and other troubles
Strawberry Entertainment
Deep, In the  Forest
Satur Entertainment
Apex Aim Trainer
SharkFishFish
Classic Snake Adventures
Crazysoft Limited
Castle Commander
Barry McCabe
The Political Machine 2020
Stardock Entertainment
Speed Masters ASD
H.D.A.
Bloodholic
釉里红
Pulstario
Fred Wood
My hot beach vacation
Ero Team Triangle
Cats are Liquid - A Better Place
Last Quarter Studios
Country Road VR
Somun Immersive
Elder's Choice, Inc.
Ground Zero Texas - Nuclear Edition
Screaming Villains
GLOWCOMA: chapter 1
Tsvetomir Georgiev
Creature Card Idle
Fordesoft
Coffee Rush
Anarchy Enterprises
Cover Your Eyes
StarSkipp
Mini Island
MACKINN7
Classic Sudoku
Studio Goya
Wicked Willow
T.F. Wright
Hentai Pazu
GentleDriver
豊咲
Blaze Revolutions
Little Chicken Game Company
InfiniteBeat
CyberGods
FULLCHOKE : Clay Shooting VR
ROLLINGJ GAMES
Incredible Dracula: Witches' Curse
Platinum Games
Vesper
Cordens Interactive
NARWHAR Project Hornwhale
A

Welcome to Primrose Lake
SQRT3
Ghost Dimension
Saulo Rosart
Shot the Body
Medic-Games
Spooky Night 2
Aureal Studio
XP Girls
OneShark
Pixel Ripped 1995
ARVORE Immersive Experiences
MazM: Jekyll and Hyde
Growing Seeds
BATYA
OKONNAYA RAMA
Somewhere inside
Funny Games
Stirring Abyss
Sleepy Sentry
K-Project
ICBM
SoftWarWare
K-Project
Ultimate Visualizer
UltimateDesktop
Dezatopia
HEY
Picross.io
Alexander Mirgorod
名媛女友-Girlfriend invites
Flyboy
Escape the Ayuwoki
Deadlycrow Games
The Summoning
Randumb Studios
Yu Escape / Monday
Team Tailnut
Stream Fighters
Cringe Engine Studio
Deadly Night - No Escape
SilverDam™
Molly: fear of clowns
BA.D
江湖余生：缘起
柚子丁工作室
邻居大叔/UncleNeighbor:uncle Dating Simulator
GameUncle
Dangerous Blaster
Juicy Frog
GASH
Jaden Duchene
DIY
Katana Kata
Stas Shostak
Alexander Sitnikov
Bolf
SimTenero
VECTORLORD
NerveQuake Software Inc.
TRANSFORMERS: BATTLEGROUNDS
Coatsink
Gigantosaurus The Game
WildSphere
PONG Quest™
Chequered Ink Ltd.
Geometry Hero
Kyo Soft
The Sorrowvirus
Watch

Steve O'Gorman
Street Jam: The Rise
Hosted Games
Ailment
Ivan Panasenko
KiKi's adventure
PhilippeStudio
VR MEDIA VIEWER
BoninblueDesignLaboratory
Teardown
Tuxedo Labs
Femida
Art Interactive
Context
SunshineOvercast
clutchball
Sokpop Collective
Rival Stars Horse Racing: Desktop Edition
PikPok
Hentai Nekogirl
GentleDriver
楓野
Desert coachman
迈克儿
Guilty Parade
Nozori Games
Tank Impact
MiCake
Live Desktop
chengcong
Nanoworld
Kedexa
Best Plumber
High and Wide
The Fog Knows Your Name
Choice of Games
nothing & nowhere
ebi-hime
Centralia: Homecoming
indie_games_studio
Jinga Online
2SD
Grand Academy II: Attack of the Sequel
Choice of Games
Breach: The Archangel Job
Hosted Games
Gods of Love: An Otome Visual Novel
Y Press Otomes
SisterFight
FoxNyan Games
Fighting Moore
complicatedAloofness
1812: Napoleon Wars
First Games Interactive
Asian Riddles
Creobit
Mahjong Gold
Creobit
Death and Taxes
Placeholder Gameworks
Hentai Shiri
GentleDriver
トマ
フキタ
Alexis Almighty: Daughter of Hercules
Jetdogs Studio

Nic Schubert
Nebuchadnezzar
Nepos Games
GEESE vs CTHULHU
Anatoliy Loginovskikh
Last Shark Standing
Shark Fin Games
Hexagun
GraphXGames
Hentai Killer
PlatSTUDIO
Temply Girls
Red-Pixel
Heroes Of Avranche
Kumova Slama
Morels: The Hunt
Abrams Studios
A NIGHTMARE'S TRIP
SKY HOUR WORKS
Cube Attack
Pug Studio
Hero Of The Forest
MediocreArc
Balls!🤬🍆
butcha
Swinger-Man
Andreas Gullberg Larsen
Holy Knight Luviria
Daijyobi Institute
Teragard
Michael Squirrel
Channel Player
Applet LLC
Output Pasture
DuskDogStudio
Harakatsu 2
INTERHEART glossy
Help! I am REALLY horny!
Professional Villains
Treasure Tomb VR
LA12 Studios
HENTAI EXOTICA vol.2
Elementarium games
Princess Maker ~Faery Tales Come True~ (HD Remake)
CFK Co., Ltd.
Hentai Eroshojo
GentleDriver
MistyNight
Raindancer
Strange Creatures Studio LLC
Spectating Simulator The Racing
Sensei Games
Girl Kill Zombies
GK
DOOMBRINGER
Anomic Games
Safari Grounds - The Wilpattu Leopard
Upulanka Premasiri
VirtuaLiron - Immersive YOGA practice
Oshri
The Exorc

...

Echo Tokyo: Phoenix
Echo Games
Break Chance Memento
Cyanide Tea
Silver Knight
Wojciech Krupinski
Discouraged Workers TEEN
YGGDRASIL STUDIO
Chef Solitaire: USA
The Revills Games
The Revenant Prince
Nomina Games
Axe, Bow & Staff
Clewcat Games
UnderDread
Bigzur Games
Soviet City
chickeninthecornstudio
Tiltagon
Kiemura Ltd
No Seat?
khos85
Rytmik Lite Chiptune Synthesizer
CINEMAX, s.r.o.
Epic Battle Fantasy 5
Matt Roszak
Dungeons Are Random
PIXELBUG
The Legend: A University Story
Austin Handle
Diorama No.3 : The Marchland
The Shoebox Diorama (Daniel Ernst)
Mazement
Wronghut
Alchemist's Awakening
Osaris Games
Midnight
Petite Games
Inside The Code
Tinycorp
Clash of the Monsters
Cadence Games
Mystic Destinies: Serendipity of Aeons
Aeon Dream Studios
Frosty Kiss
2Chance Projects
IIchan Eroge Team
Breakroom
MURE VR
Earn to Die 2
Toffee Games
Automobilista
Reiza Studios
Hubert's Island Adventure: Mouse o' War
Cheese and Bacon Games
Action Legion
Aeonic Entertainment
Doodle Kingdom
JoyBits Ltd.
Ph

Astervoid 2000
Mad Capacity
LREVG
Carpe Diem
Moonlit Works
Gods and Idols
Viking Tao
Dinocide
AtomicTorch Studio
Deadlight: Director's Cut
Tequila Works
Abstraction Games
Project Pulsation
VGstudio
Crudelis
Whiskey Jack Games
War of Rights
Campfire Games
Fragmental
Ruffian Games
The Glow
Brainwashing games
Celestrion
dSky9, Inc.
The fall of gods
GeexGames
Choice of Kung Fu
Choice of Games
Air Missions: HIND
3Division
Day D: Tower Rush
Creobit
Project Highrise
SomaSim
The Little Acre
Pewter Games Studios
Adelantado Trilogy. Book one
Whiterra
Where's My Mommy?
Exquisite Games
HERO DEFENSE
Happy Tuesday
Summer Fling
Dharker Studio
Apocalypse Hotel - The Post-Apocalyptic Hotel Simulator!
Michael Flynn
Invasion: Brain Craving
New Reality Games
Color Chaos
Daniel Riley
Hyper Gods
Ground Control Studios
Save Your Mother
Tuomo Laine
Gardenarium
Paloma Dawkins
Kyler Kelly
Catherine Debard
Hit Tank PRO
Bacq Stellan
Our Love Will Grow
John Wizard
Community College Hero: Trial by Fire
Hosted Games

DEDstress
3StrakGames
Chinatris
仓鼠动力
Args Studio
Recyclomania
PixQuake
TILTit
MABE Play
CRUEL WORLD
SiberianGames
The Dark Eye : Book of Heroes
Random Potion Oy
Chapeau
Salt Castle Studio
Dictators:No Peace Countryballs
RPN Indie Developer
Two Weeks in Painland
Unusual Games
Cetetorius
VI.SH Games
AMazing TD
Go4 Entertainment
Tractor Cargo Driving Simulator
Boogygames Studios
HalloweenStory
Dnovel
Ven VR Adventure
Monologic Games
THE ART - Metamorphosis
Life Jumb DT
Mask of Sanity
UNICORE Games Studios
Marburgh
Marcotte
Pixamal Zoo
Monkey Maneuvers
Dark Elf's Adventure
Uzura Studio
Super Platformer Gun
Expresso Studios
Biomass
Final Scene dev
B.i.t.Lock
Petorio
Peter Hofbauer
Hentai Evilgirls
GentleDriver
湖畔
Demolition Expert - The Simulation
abramedia
An Egg Can Dream
Floor Chicken
BioMech
ERMedia
Figure Simulator War
H-GIRL
Workhard
Red twice potato
Slime Adventure Legacy
Vlad Dunaev
I LIKE THE FLOWERS
Red twice potato
Morbolbo: Enter the Maze
Laughing Tower
AREA 51 - DEFENCE
SRS Gam

Rezist: Tower Defense
SparkGames
Weeping Willow - Detective Visual Novel
TechnoCat Games
Affinity
Bold Brush Games
Cross Country Express
Oddfellows Software
Roots Of The Woods
PlumBear Games
DOUBLE INVASION!!
Goose Ladder Games
Plunge
Spooky Buns
Use Your Brain!
Dr. DreamCrusher
1001 Jigsaw World Tour: Europe
Creobit
Solitaire Victorian Picnic
Big Up Studio
live Desktop Beauty
Bad Bunny Games
Hell Firefighter
Digital Entertainment Novels
Cube World
Picroma
Zoological Era II
Jareed Dempsey
ZeroZone2020
HaDe Games
StarSystemStudios
Under: Depths of Fear
Globiss Interactive
祭壇の花/Saidan no Hana
Zacro
Saturday of Piercing Screams
derevotyan
Power Stealers
Intertum
CROSSNIQ+
FUTURE MEMORY
Achievement Simulator
Leeson
Orion13
Metro VR Studios
ZAAM
Alon Zubina
The Princess, the Stray Cat, and Matters of the Heart 2
TOKYOTOON
Whack A Hoe
Nasty Girl Gaming
IGOR MAKS The Meme Lord
SЁLO
Koi Unleashed
Wild Grip
Try again!
Antizero company
Solitaire Jack Frost Winter Adventures
Somer Games
Solitaire

Super Lee World
Kyrios Game Studio
Cruel Bands Career
heran&99
Seraphic Destroyer
Sonic-Alpha
My name is You and it's the only unusual thing in my life
Nikita Kaf
Sort Battle: Dungeon
Incandescent Workshop LLC
Rituals in the Dark
ebi-hime
Sunflower
Kory Toombs
FRENZY PLANTS
MADSKILLZGAME
Robot terminator
tina
Engineer Arena
WaveParadigm
Nation Breakers: Steam Arena
Coded Clay Interactive
Meegah Mem 2
Crapcom
Discord Bot Studio
Discord Bot Studio
GLADIUM
Lorebound Entertainment
Wonder Blade 惊奇剑士
PUPPET DEPOT GAME STUDIO
Monochrome Order
Hit-Point Co.,Ltd.
Asdivine Kamura
Exe Create Inc.
Wizards of Brandel
Exe Create Inc.
Revenant March
Strandline Games
Terminal squad: Sentinel
Droid Riot
1976 - Back to midway
Ivanovich Games
Seven Wonders of St. Clementine
Imagine Fantasia
Dead Hand
One Man Army Games
Theater Commander: The Coming Wars, Modern War Game
Sand Table Software
G.A.C.K. - Gaming App Construction Kit
Gernot Frisch
Learn Spanish! Easy Vocabulary
Dr. Cyril Splutterworth
Day of v

In [28]:
#session.rollback()